<a href="https://colab.research.google.com/github/JsuccessJ/dialect_transformer/blob/main/basic_fine_tuning%2Bclassification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
print(torch.cuda.is_available())

True


In [ ]:
#설치 후 런타임 재실행해야함
!pip install accelerate>=0.20.1


In [ ]:
!pip install transformers

In [ ]:
# !pip install wandb

In [ ]:
# import wandb
# !wandb login "f2d7cd4fedd2822e534395005a236ac7b01e0036"
# wandb.init(project="dialect_transformer")

In [ ]:
# wandb.run.name = '분류기 추가'
# wandb.run.save()

In [ ]:
# Hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 10

args = {
    "learning_rate": learning_rate,
    "epochs": epochs,
    "batch_size": batch_size
}
# wandb.config.update(args)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader


In [ ]:

# Load CSV data
data = pd.read_csv('/content/drive/MyDrive/hjse/jaesung/dataset_경상도1만+전라도1만_train.csv')

# 문장과 라벨 분리
X = data['dialect'].values
y = data['class'].values

# 라벨 인코딩 (0 or 1)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

# 결과를 dense array로 변환
X_tfidf_array = X_tfidf.toarray()

# array를 float32로 형변환
X_tfidf_float = X_tfidf_array.astype('float32')


In [ ]:
X_tfidf_float

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
import torch

X_tensor = torch.tensor(X_tfidf_float)
y_tensor = torch.tensor(y)

In [ ]:
X_tensor

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
y_tensor

tensor([0, 0, 0,  ..., 1, 1, 1])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
import torch.nn as nn

class DialectClassifier(nn.Module):
    def __init__(self, input_dim):
        super(DialectClassifier, self).__init__()
        self.layer = nn.Linear(input_dim, 2)

    def forward(self, x):
        return self.layer(x)

model = DialectClassifier(X_tensor.shape[1])


In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for batch in dataloader:
        X_batch, y_batch = batch
        optimizer.zero_grad()
        output = model(X_batch.float())
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()


In [ ]:
torch.save(model.state_dict(), 'dialect_classifier.pth')
